In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd
import uuid

In [3]:
year =2022
month = 2
input_file = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet'
output_file = 'output/yellow_ride_id_prediction.parquet'
with open('/home/ubuntu/mlops-zoomcamp/04-deployment/homework4/models/lin_reg.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/ubuntu/anaconda3/envs/homework/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ubuntu/anaconda3/envs/homework/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:


def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    dicts = df[categorical].to_dict(orient='records')
    
    
    return df, dicts

In [5]:
def make_predictions():  
    df, dicts = read_data(input_file)  
    X_val = dv.transform(dicts)
    y_pred = model.predict(X_val)
    return y_pred

In [6]:
def new_dataframe(year, month):
    df, dicts = read_data(input_file)
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['prediction'] = make_predictions()
    print(df_result.prediction.std())
    df_result.to_parquet(output_file, engine='pyarrow', compression=None, index=False)
    return output_file

In [7]:
new_dataframe(2022, 2)

In [ ]:
# import os
# filename = 'lin_reg.bin'

# file_path = os.path.abspath(filename)
# print(file_path)

/home/ubuntu/mlops-zoomcamp/04-deployment/homework4/lin_reg.bin
